In [3]:
import mwclient
import time

site = mwclient.Site('en.wikipedia.org')
page = site.pages['Bitcoin']

In [4]:
revs = list(page.revisions())

In [5]:
revs[0]

OrderedDict([('revid', 1154294052),
             ('parentid', 1154267211),
             ('minor', ''),
             ('user', 'Finnusertop'),
             ('timestamp',
              time.struct_time(tm_year=2023, tm_mon=5, tm_mday=11, tm_hour=14, tm_min=25, tm_sec=53, tm_wday=3, tm_yday=131, tm_isdst=-1)),
             ('comment', 'cs1')])

In [ ]:
revs = sorted(revs, key=lambda rev: rev["timestamp"]) 

In [ ]:
revs[0]

In [ ]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

def find_sentiment(text):
    sent = sentiment_pipeline([text[:1000]])[0]
    score = sent["score"]
    if sent["label"] == "NEGATIVE":
        score *= -1
    return score

In [ ]:
edits = {}

for rev in revs:        
    date = time.strftime("%Y-%m-%d", rev["timestamp"])
    if date not in edits:
        edits[date] = dict(sentiments=list(), edit_count=0)
    
    edits[date]["edit_count"] += 1
    
    comment = rev.get("comment", "")
    edits[date]["sentiments"].append(find_sentiment(comment))

In [ ]:
from statistics import mean

for key in edits:
    if len(edits[key]["sentiments"]) > 0:
        edits[key]["sentiment"] = mean(edits[key]["sentiments"])
        edits[key]["neg_sentiment"] = len([s for s in edits[key]["sentiments"] if s < 0]) / len(edits[key]["sentiments"])
    else:
        edits[key]["sentiment"] = 0
        edits[key]["neg_sentiment"] = 0
    
    del edits[key]["sentiments"]

In [ ]:
import pandas as pd

edits_df = pd.DataFrame.from_dict(edits, orient="index")

In [ ]:
edits_df

In [ ]:
edits_df.index = pd.to_datetime(edits_df.index)

In [ ]:
from datetime import datetime

dates = pd.date_range(start="2009-03-08",end=datetime.today())

In [ ]:
edits_df = edits_df.reindex(dates, fill_value=0)

In [ ]:
edits_df

In [ ]:
rolling_edits = edits_df.rolling(30, min_periods=30).mean()

In [ ]:
rolling_edits = rolling_edits.dropna()

In [ ]:
rolling_edits

In [ ]:
rolling_edits.to_csv("wikipedia_edits.csv")